# Feature Engineering - Zipcodes

In [1]:
# installing the requirements:
!pip install -r ../configs/dependencies/dataprep_requirements.txt >> ../configs/dependencies/package_installation.txt

In [2]:
# loading the magic commands:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [3]:
###### Loading the necessary libraries #########

# PySpark dependencies:s
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
import pyspark.sql.types as T
from pyspark.sql.window import Window

# Sedona dependencies:
from sedona.utils.adapter import Adapter
from sedona.register import SedonaRegistrator
from sedona.utils import KryoSerializer, SedonaKryoRegistrator
from sedona.core.SpatialRDD import SpatialRDD
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.formatMapper import GeoJsonReader

# database utilities:
from sqlalchemy import create_engine
import sqlite3 as db
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import fiona

# plotting and data visualization:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, Image

# other relevant libraries:
import warnings
import unidecode
import inflection
import unicodedata
from datetime import datetime, timedelta
from functools import partial
import json
import re
import os
from glob import glob
import shutil
import itertools
import chardet

# importing the atlas utilities:
from atlasutils import (
    save_to_filesystem,
    save_as_table,
    rotate_xticks,
    get_file_encoding,
    normalize_entities,
    normalize_column_name,
    apply_category_map,
    standardize_variable_names,
    get_null_columns,
    replace_decimal_separator,
    convert_to_geopandas,
    drop_invalid_census_columns,
    clean_census_column_name,
    get_file_crs,
    get_column_values,
    load_geospatial_file,
    convert_geometry,
    rank_feature,
    save_geospatial_file,
)


# setting global parameters for visualizationsss:
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

# 0. Configuring Spark

In [4]:
# function to encapsulate standard spark configurations:
def init_spark(app_name):

    spark = (
        SparkSession.builder.appName(app_name)
        .config("spark.files.overwrite", "true")
        .config("spark.serializer", KryoSerializer.getName)
        .config("spark.kryo.registrator", SedonaKryoRegistrator.getName)
        .config(
            "spark.jars.packages",
            "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,"
            "org.datasyslab:geotools-wrapper:geotools-24.1",
        )
        .config("spark.sql.repl.eagerEval.enabled", True)
        .config("spark.sql.repl.eagerEval.maxNumRows", 5)
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
        .config("spark.sql.parquet.compression.codec", "gzip")
        .config("sedona.global.charset", "utf8")
        .config("sedona.global.index", "true")
        .enableHiveSupport()
        .getOrCreate()
    )

    SedonaRegistrator.registerAll(spark)

    return spark

In [5]:
# init the spark session:
spark = init_spark("SP Atlas - Zipcode-level Feature Engineering")

In [6]:
# verifying the session status:
spark

# 1. Loading and Inspecting the Data

## 1.1 Additional Layers

In [7]:
# loading the raw dataset:
LAYERS_DATA_DIR = "../data/raw/sp_layers/"
RAW_DATA_DIR = "../data/raw/"

### 1.1.1 Parks

In [8]:
# loading the park data:
df_parks = load_geospatial_file(LAYERS_DATA_DIR + "parks/*", spark)

# convert the geometry type:
df_parks = convert_geometry(df_parks, "geometry", "epsg:29193", "epsg:4326")

In [9]:
# improving column names:
df_parks = df_parks.select(
    F.col("pq_id").alias("park_id"),
    F.col("pq_area").alias("park_area"),
    F.col("pq_nome").alias("park_name"),
    F.col("pq_PrefReg").alias("park_subprefecture"),
    F.col("geometry"),
)

### 1.1.2 Favelas

In [10]:
# loading the park data:
df_favelas = load_geospatial_file(LAYERS_DATA_DIR + "favelas_macro/*", spark)

# convert the geometry type:
df_favelas = convert_geometry(df_favelas, "geometry", "epsg:29193", "epsg:4326")

In [11]:
# fixing the columns:
new_cols = ["geometry", "favela_id", "favela_area", "favela_name"]

df_favelas = df_favelas.drop("DATULTATZ", "ENDERECO", "NOMESEC", "PROPRTR")

original_cols = df_favelas.columns
cols_map = dict(zip(original_cols, new_cols))

for col, new_col in cols_map.items():
    df_favelas = df_favelas.withColumnRenamed(col, new_col)

### 1.1.3 Train and Metro

In [12]:
df_train = load_geospatial_file(LAYERS_DATA_DIR + "train/*", spark)
df_train = convert_geometry(df_train, "geometry", "epsg:29193", "epsg:4326")

# cleaning columns:
df_train = df_train.select(F.col("geometry"), F.col("etr_nome").alias("station_name"))

In [13]:
df_metro = load_geospatial_file(LAYERS_DATA_DIR + "metro/*", spark)
df_metro = convert_geometry(df_metro, "geometry", "epsg:29193", "epsg:4326")

# cleaning columns:
df_metro = df_metro.select(F.col("geometry"), F.col("emt_nome").alias("station_name"))

In [14]:
# union all dataframes:
df_rail = df_train.union(df_metro)

In [15]:
# verifying the number of resulting records:
df_rail.count()

196

## 1.2 Zipcode Features

In [16]:
# Data prep results:
PROCESSED_DATA_DIR = "../data/processed/"

# reading the result from the data prep:
df_zipcode_census = spark.read.parquet(
    PROCESSED_DATA_DIR
    + "sp_census/units_of_interest/zipcodes/tb_zipcode_census.parquet"
)

df_zipcode_iptu = spark.read.parquet(
    PROCESSED_DATA_DIR + "sp_iptu/tb_zipcode_iptu_no_geo.parquet"
)

df_zipcode_iptu = df_zipcode_iptu.withColumnRenamed("zipcode_zipcode", "zipcode")

In [17]:
# joinining the district base features:
df_zipcode = df_zipcode_census.join(
    df_zipcode_iptu,
    how="left",
    on=["zipcode"],
)

In [18]:
# verifying the data integrity:
get_null_columns(df_zipcode, normalize=True)  # looks good to go

-RECORD 0------------------------------------------------------------------------------------------------------
 zipcode                                                                               | 0.0                   
 zipcode_alphabetized_population                                                       | 0.0                   
 zipcode_area_in_hectares                                                              | 0.0                   
 zipcode_average_age_household_leaders                                                 | 0.0                   
 zipcode_average_age_women_household_leaders                                           | 0.0                   
 zipcode_average_household_income                                                      | 0.0                   
 zipcode_average_income_household_leaders                                              | 0.0                   
 zipcode_average_income_women_household_leaders                                        | 0.0            

# 2. Generating Layer-based Features

In [19]:
# registering dataframes to sql context:
df_zipcode.createOrReplaceTempView("tb_zipcode")
df_rail.createOrReplaceTempView("tb_stations")
df_parks.createOrReplaceTempView("tb_parks")
df_favelas.createOrReplaceTempView("tb_favelas")

## 2.1 Parks
Considering Parks as a feature takes into account two situations:
1. A region of any kind *containing* or *crossing* a park;
2. The distance of a region to the closest park; 

Since we are comparing polygons to polygons, it doesn't make much sense calculating the distance between them. Instead, we will summarize their units such that we can compare the distance between two points. In this case, we can use their centroids. 

> *Note: As of writing this notebook (July 4th, 2021), writing KNN queries (which is supported by Sedona's RDD api) can be made, but I will keep this for a later iteration of this project, as I am not that comfortable with the RDD api.*

In [20]:
# adding the centroid and area of the polygons:
Q_PARKS = """
with crossings as (
    SELECT 
        A.zipcode,
        B.park_name,
        B.park_area
    FROM tb_zipcode as A, tb_parks as B
    WHERE ST_Crosses(B.geometry, A.geometry)
),

touches as (
    SELECT 
        A.zipcode,
        B.park_name,
        B.park_area
    FROM tb_zipcode as A, tb_parks as B
    WHERE ST_Touches(B.geometry, A.geometry)
),

full_union as (
    SELECT * FROM crossings
    UNION
    SELECT * FROM touches    
)

SELECT
    DISTINCT zipcode,
    park_name,
    park_area
FROM full_union
"""

# matching the areas of ponderation to their sectors:
df_parks_match = spark.sql(Q_PARKS)

In [21]:
# grouping on district level to count the number of parks:
df_zipcode_parks = df_parks_match.groupby("zipcode").agg(
    F.countDistinct(F.col("park_name")).alias("n_parks"),
    F.sum(F.col("park_area")).alias("total_park_area"),
)

## 2.2 Favelas
To address Favelas as a feature, we have the same criteria as the case of Parks.

In [22]:
# adding the centroid and area of the polygons:
Q_FAVELAS = """
with crossings as (
    SELECT 
        A.zipcode,
        B.favela_name,
        B.favela_area
    FROM tb_zipcode as A, tb_favelas as B
    WHERE ST_Crosses(B.geometry, A.geometry)
),

touches as (
    SELECT 
        A.zipcode,
        B.favela_name,
        B.favela_area
    FROM tb_zipcode as A, tb_favelas as B
    WHERE ST_Touches(B.geometry, A.geometry)
),

full_union as (
    SELECT * FROM crossings
    UNION
    SELECT * FROM touches    
)

SELECT
    DISTINCT zipcode,
    favela_name,
    favela_area
FROM full_union
"""

# matching the areas of ponderation to their sectors:
df_favelas_match = spark.sql(Q_FAVELAS)

In [23]:
# grouping on district level to count the number of parks:
df_zipcode_favelas = df_favelas_match.groupby("zipcode").agg(
    F.countDistinct(F.col("favela_name")).alias("n_favelas"),
    F.sum(F.col("favela_area")).alias("total_favela_area"),
)

# 3. Preparing the Layer-based features in the Neighborhood level

In [24]:
# joining layer-based features to the main dataset:
df_zipcode_features = df_zipcode.join(df_zipcode_parks, on=["zipcode"], how="left")

df_zipcode_features = df_zipcode_features.join(
    df_zipcode_favelas, on=["zipcode"], how="left"
)

# dropping duplicates:
df_zipcode_features = df_zipcode_features.drop_duplicates(subset=["zipcode"])

In [25]:
# fill null values in the new columns:
new_columns = ["n_parks", "total_park_area", "n_favelas", "total_favela_area"]

df_zipcode_features = df_zipcode_features.fillna(0, subset=new_columns)

In [26]:
# adding a prefix for the level of interest to the new columns:
for col in new_columns:
    df_zipcode_features = df_zipcode_features.withColumnRenamed(col, f"zipcode_{col}")

# 4. Ranking-based columns
While it is valuable to have the aggregated features as raw values, it is also quite useful to have rank-based features, as they can be effectively a way to represent the categorical entities (the `district`, for example) directly as some sort of "embedding". I will generate some key features using this principle next using the `rank_feature` helper function, which, by default, transforms features into their `percent_rank` representation.

In [27]:
# adding rank on literacy rate -> the lowest literacy rate, the lower the rank
df_zipcode_features = rank_feature(df_zipcode_features, "zipcode_literacy_rate")

In [28]:
# adding rank on monthly income
df_zipcode_features = rank_feature(
    df_zipcode_features, "zipcode_average_monthly_income"
)

In [29]:
# adding rank on per capita income
df_zipcode_features = rank_feature(
    df_zipcode_features, "zipcode_average_per_capita_income"
)

In [30]:
# adding rank on population (raw)
df_zipcode_features = rank_feature(df_zipcode_features, "zipcode_population")

In [31]:
# adding rank on lot and construction square meter valuess
df_zipcode_features = rank_feature(
    df_zipcode_features, "zipcode_average_lot_square_meter_value"
)

df_zipcode_features = rank_feature(
    df_zipcode_features, "zipcode_average_construction_square_meter_value"
)

In [32]:
# adding rank on property_age -> the newer the properties, the higher the rank
df_zipcode_features = rank_feature(
    df_zipcode_features, "zipcode_average_property_age", ascending=False
)

In [33]:
# persisting the dataset to avoid heap space problems:
df_zipcode_features.cache()

zipcode,zipcode_alphabetized_population,zipcode_area_in_hectares,zipcode_average_age_household_leaders,zipcode_average_age_women_household_leaders,zipcode_average_household_income,zipcode_average_income_household_leaders,zipcode_average_income_women_household_leaders,zipcode_average_ipvs_factor_1,zipcode_average_ipvs_factor_1_rural,zipcode_average_ipvs_factor_2,zipcode_average_ipvs_factor_2_rural,zipcode_average_monthly_income,zipcode_average_number_children,zipcode_average_per_capita_income,zipcode_average_permanent_private_households_with_10_or_more_residents,zipcode_average_permanent_private_households_with_1_resident,zipcode_average_permanent_private_households_with_2_residents,zipcode_average_permanent_private_households_with_3_residents,zipcode_average_permanent_private_households_with_4_residents,zipcode_average_permanent_private_households_with_5_residents,zipcode_average_permanent_private_households_with_6_residents,zipcode_average_permanent_private_households_with_7_residents,zipcode_average_permanent_private_households_with_8_residents,zipcode_average_permanent_private_households_with_9_residents,zipcode_average_permanent_private_households_with_electricity,zipcode_average_permanent_private_households_without_exclusive_use_bathroom_residents,zipcode_average_permanent_private_houses_fully_owned,zipcode_average_permanent_private_houses_in_acquisition,zipcode_average_permanent_private_properties_fully_owned,zipcode_average_permanent_private_properties_in_acquisition,zipcode_average_permanent_private_properties_rented,zipcode_average_permanent_private_rented_properties,zipcode_average_population_density,zipcode_average_prop_households_access_water,zipcode_average_proportion_households_electricity,zipcode_average_proportion_households_no_income,zipcode_average_proportion_households_one_halft_sm_income,zipcode_average_proportion_households_sanitation,zipcode_average_proportion_households_trash_collection,zipcode_average_proportion_households_two_or_more_income,zipcode_average_proportion_households_two_sm_income,zipcode_average_proportion_househols_one_eigth_sm_income,zipcode_average_proportion_income_participation_in_household_responsible,zipcode_average_proportion_literate_people,zipcode_average_proportion_literate_women,zipcode_average_proportion_population_up_to_5_years_age,zipcode_average_proportion_private_households_one_half_sm_income,zipcode_average_proportion_private_households_one_quarter_sm_income,zipcode_average_proportion_responsible_people_up_to_30_years,zipcode_average_proportion_women_up_to_30_years,zipcode_average_residents_in_permanent_households,zipcode_average_residents_per_household,zipcode_avg_ipvs_group,zipcode_households,zipcode_improvised_households,zipcode_literacy_rate,zipcode_max_ipvs_group,zipcode_min_ipvs_group,zipcode_n_regions_high_social_vulnerability,zipcode_n_regions_low_social_vulnerability,zipcode_n_regions_medium_social_vulnerability,zipcode_number_house_workers,zipcode_number_subnormal_aglomerates,zipcode_permanent_housholds,zipcode_permanent_private_households_with_10_or_more_residents,zipcode_permanent_private_households_with_1_resident,zipcode_permanent_private_households_with_2_residents,zipcode_permanent_private_households_with_3_residents,zipcode_permanent_private_households_with_4_residents,zipcode_permanent_private_households_with_5_residents,zipcode_permanent_private_households_with_6_residents,zipcode_permanent_private_households_with_7_residents,zipcode_permanent_private_households_with_8_residents,zipcode_permanent_private_households_with_9_residents,zipcode_permanent_private_households_with_electricity,zipcode_permanent_private_households_with_up_to_3_bathrooms,zipcode_permanent_private_households_without_exclusive_use_bathroom_residents,zipcode_permanent_private_houses_fully_owned,zipcode_permanent_private_houses_in_acquisition,zipcode_permanent_private_properties_fully_owned,zipcode_permanent_private_properties_in_acquisition,zipcode_permanent_private_properties_rented,

In [34]:
# removing the other datasets:
del df_parks
del df_favelas
del df_zipcode
del df_rail

# 5. Exporting Neighborhood Features

In [35]:
# save the results to the specified directory:
ZIPCODE_OUTPUT = f"zipcode/tb_zipcode"
FEATURE_UNITS_OF_INTEREST = "../data/feature/units_of_interest/"

df_zipcode_features = df_zipcode_features.drop("zipcode_centroid", "district_centroid")

save_geospatial_file(
    df_zipcode_features,
    FEATURE_UNITS_OF_INTEREST + ZIPCODE_OUTPUT + ".parquet",
)

True

In [36]:
# saving the results without geometries:
ZIPCODE_NO_GEO = f"zipcode/tb_zipcode_no_geo"

df_zipcode_no_geo = df_zipcode_features.drop("geometry")

save_to_filesystem(
    df_zipcode_no_geo,
    FEATURE_UNITS_OF_INTEREST,
    ZIPCODE_NO_GEO,
    ZIPCODE_NO_GEO + ".parquet",
)

True